In [1]:
import importlib, chan
ok = True
for name in ["DEFAULT_POLICY","_map_beichi_to_1st","generate_mmd2_from_mmd1",
             "generate_mmd3_from_zs","merge_mmd_signals","DEFAULT_MERGE_CONFIG","generate_trading_signals"]:
    try:
        getattr(chan, name)
    except Exception as e:
        ok = False
        print(f"[FAIL] missing: {name} -> {e}")

import pandas as pd
from chan import merge_mmd_signals
df = merge_mmd_signals(pd.DataFrame([]), pd.DataFrame({"side":["buy"]}))
ok &= (list(df.columns)[0] == "ts" and df.empty)

print("signals export & merge_mmd_signals health:", "OK" if ok else "FAILED")


signals export & merge_mmd_signals health: OK


In [1]:
from pathlib import Path
import pandas as pd
from strategy.baselines_traditional import (
    macd_signals, rsi_signals, boll_signals, moving_average_signals,
    merge_traditional_signals, simple_backtest
)

DATA_ROOT = Path("/Users/cc/PycharmProjects/ChanLab/data")
SYMBOL, TF = "BTCUSDT", "5m"
kline = pd.read_csv(DATA_ROOT/SYMBOL/f"{TF}.csv", parse_dates=["timestamp"])

sig = merge_traditional_signals(
    macd_signals(kline),
    rsi_signals(kline),
    boll_signals(kline),
    moving_average_signals(kline, fast=10, slow=30),  # 放最后以提高优先级
)
res = simple_backtest(sig, kline, fee=0.001, slippage=0.0)
print({k:v for k,v in res.items() if k!='curve'})

{'trades': 6142, 'win_rate': 0.28573754477368934, 'ret': -0.9981195139639117, 'sharpe': -22.2912063109565, 'mdd': -0.9981215451621338}


In [21]:
from pathlib import Path
import pandas as pd

# 传统 baseline
from strategy.baselines_traditional import (
    macd_signals, rsi_signals, boll_signals, moving_average_signals,
    merge_traditional_signals, simple_backtest as bt_trad
)

# ML baseline
from strategy.baselines_ml import ml_signals_from_kline, simple_backtest as bt_ml

DATA_ROOT = Path("/Users/cc/PycharmProjects/ChanLab/data")
SYMBOL, TF = "BTCUSDT", "5m"
kline = pd.read_csv(DATA_ROOT/SYMBOL/f"{TF}.csv", parse_dates=["timestamp"])

# 传统四件套
sig_trad = merge_traditional_signals(
    macd_signals(kline),
    rsi_signals(kline),
    boll_signals(kline),
    moving_average_signals(kline, fast=10, slow=30)  # 放最后：同刻冲突优先MA
)
res_trad = bt_trad(sig_trad, kline, fee=0.001, slippage=0.0)

# ML 多模型（任选其一先跑）
sig_rf  = ml_signals_from_kline(kline, model_name="rf", model_params={"n_estimators":300, "max_depth":8})
sig_lr  = ml_signals_from_kline(kline, model_name="lr", model_params={"C":0.5, "max_iter":400})
sig_gb  = ml_signals_from_kline(kline, model_name="gb")
sig_svm = ml_signals_from_kline(kline, model_name="svm", model_params={"C":1.0, "kernel":"rbf"})

res_rf  = bt_ml(sig_rf,  kline, fee=0.001, slippage=0.0)
res_lr  = bt_ml(sig_lr,  kline, fee=0.001, slippage=0.0)
res_gb  = bt_ml(sig_gb,  kline, fee=0.001, slippage=0.0)
res_svm = bt_ml(sig_svm, kline, fee=0.001, slippage=0.0)

# 简易对比表
import pandas as pd
rows = []
def pick(name, r): rows.append(dict(model=name, trades=r['trades'], win=r['win_rate'], ret=r['ret'], sharpe=r['sharpe'], mdd=r['mdd']))
pick("Traditional(MACD/RSI/BOLL/MA)", res_trad)
pick("ML-RF",  res_rf)
pick("ML-LR",  res_lr)
pick("ML-GB",  res_gb)
pick("ML-SVM", res_svm)
print(pd.DataFrame(rows).sort_values("sharpe", ascending=False))

                           model  trades       win       ret     sharpe  \
1                          ML-RF     477  0.559748 -0.022317   0.027465   
4                         ML-SVM     393  0.526718 -0.153260  -0.660943   
2                          ML-LR     134  0.582090 -0.224873  -0.861998   
3                          ML-GB     963  0.474559 -0.503309  -2.707784   
0  Traditional(MACD/RSI/BOLL/MA)    6142  0.285738 -0.998120 -22.291206   

        mdd  
1 -0.220553  
4 -0.236629  
2 -0.327491  
3 -0.524916  
0 -0.998122  


In [14]:
import importlib
import strategy.baselines_ml as ml
importlib.reload(ml)

<module 'strategy.baselines_ml' from '/Users/cc/PycharmProjects/ChanLab/strategy/baselines_ml.py'>

In [1]:
import pandas as pd
from itertools import product
from strategy.baselines_ml import ml_signals_from_kline, simple_backtest

def grid_search_ml(kline: pd.DataFrame,
                   model_grid=("rf","lr","svm","gb"),
                   p_buy_list=(0.60,0.62,0.65),
                   p_sell_list=(0.40,0.38,0.35),
                   gap_list=(60,120,240),
                   model_params_map=None,
                   train_bars=2000, test_bars=500,
                   label_horizon=3, label_eps=0.001):
    rows = []
    model_params_map = model_params_map or {
        "rf": {"n_estimators":300, "max_depth":8},
        "lr": {"C":1.0, "max_iter":2000, "class_weight":"balanced"},
        "svm":{"C":1.0, "kernel":"rbf", "gamma":"scale", "class_weight":"balanced"},
        "gb": {}
    }
    for mdl, pb, ps, gap in product(model_grid, p_buy_list, p_sell_list, gap_list):
        if pb <= ps:  # 合理边界
            continue
        sigs = ml_signals_from_kline(
            kline,
            model_name=mdl,
            model_params=model_params_map.get(mdl, {}),
            p_buy=pb, p_sell=ps,
            train_bars=train_bars, test_bars=test_bars,
            label_horizon=label_horizon, label_eps=label_eps,
            min_gap_minutes=gap
        )
        res = simple_backtest(sigs, kline, fee=0.001, slippage=0.0)
        rows.append(dict(
            model=mdl, p_buy=pb, p_sell=ps, gap=gap,
            trades=res["trades"], win=res["win_rate"],
            ret=res["ret"], sharpe=res["sharpe"], mdd=res["mdd"]
        ))
    df = pd.DataFrame(rows).sort_values(["ret","sharpe"], ascending=[False, False]).reset_index(drop=True)
    return df

